In [50]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

def im2double(im):
    """
    Converts uint image (0-255) to double image (0.0-1.0) and generalizes
    this concept to any range.

    :param im:
    :return: normalized image
    """
    min_val = np.min(im.ravel())
    max_val = np.max(im.ravel())
    
    #if (max_val - min_val) == 0:
        # create empty image!
        
    out = (im.astype('float') - min_val) / (max_val - min_val)
    return out


def make_gaussian(size, fwhm = 3, center=None):
    """ Make a square gaussian kernel.

    size is the length of a side of the square
    fwhm is full-width-half-maximum, which
    can be thought of as an effective radius.
    """

    x = np.arange(0, size, 1, float)
    y = x[:,np.newaxis]
    
    if center is None:
        x0 = y0 = size // 2
    else:
        x0 = center[0]
        y0 = center[1]

    k = np.exp(-4*np.log(2) * ((x-x0)**2 + (y-y0)**2) / fwhm**2)
    return k / np.sum(k)


def convolution_2d(img, kernel):
    """
    Computes the convolution between kernel and image

    :param img: grayscale image
    :param kernel: convolution matrix - 3x3, or 5x5 matrix
    :return: result of the convolution
    """
    # TODO write convolution of arbritrary sized convolution here
    # Hint: you need the kernelsize
    if img.ndim!=2:
        print ('convolution_2d: only 2 dim images are supported')
        return np.zeros(img.shape)
    
    offset = int(kernel.shape[0]/2)
    rk, ck = gk.shape[:2]
    ri, ci = img.shape[:2]
    
    newimg = np.zeros(img.shape)
    
    # YOUR CODE HERE
    # first implementation without border treatment, padd image at the edges
    # up to distance offset with 0, start kernel ops inside
    for r in range (offset, ri-offset): # here rows of image
        for c in range (offset, ci-offset): # here columns of image
            # slice out kernel sized section
            newimg[r,c] = \
                np.sum(img[r-offset:r+offset+1,c-offset:c+offset+1] * kernel)
            
    return newimg


if __name__ == "__main__":

    # 1. load image in grayscale
    img = cv2.imread('Lenna.png')
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 2. convert image to 0-1 image (see im2double)
    imgnorm = im2double(gray)

    # display plot in notebook
    #%matplotlib inline
    #plt.figure(figsize=(15,15))
    #plt.imshow(gray)
    #plt.show()

    # image kernels
    sobelmask_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    sobelmask_y = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
    #gk = make_gaussian(5)
    gk = make_gaussian(11)

    # check gaus matrix
    #np.set_printoptions(precision=3)
    #print(gk)
   
    # 3 .use image kernels on normalized image
    gausfiltered = convolution_2d(imgnorm, gk) 
    sobel_x = convolution_2d(gausfiltered, sobelmask_x)
    sobel_y = convolution_2d(gausfiltered, sobelmask_y)
    
    # 4. compute magnitude of gradients
    #distw = np.linalg.norm(sobel_x,sobel_y)
    mog = np.sqrt(sobel_x*sobel_x + sobel_y*sobel_y)
    
    # display plot in notebook
    #%matplotlib inline
    #plt.figure(figsize=(10,10))
    #plt.imshow(distw)
    #plt.show()

    # Show resulting images
    sobel_x_shift1 = sobel_x + 1.0
    composed_x = np.concatenate((sobel_x, sobel_x_shift1), axis=1)
    cv2.imshow("Sobel X:: Left: signed & Right: shifted to pos", composed_x)

    sobel_y_shift1 = sobel_y + 1.0
    composed_y = np.concatenate((sobel_y, sobel_y_shift1), axis=1)
    cv2.imshow("Sobel Y:: Left: signed & Right: shifted to pos", composed_y)

    cv2.imshow("mog", mog)
    cv2.waitKey(0)
    cv2.destroyAllWindows()